[View in Colaboratory](https://colab.research.google.com/github/rubencg195/PongAI/blob/master/Pong_AI.ipynb)

#Source:

https://medium.com/@dhruvp/how-to-write-a-neural-network-to-play-pong-from-scratch-956b57d4f6e0


#Problem

We are given the following:

1. A sequence of images (frames) representing each frame of the Pong game.
2. An indication when we’ve won or lost the game.
3. An opponent agent that is the traditional Pong computer player.
4. An agent we control that we can tell to move up or down at each frame.

#Solution

![The architecture Of Andrej’s solution from his blog post.](pong_assets/1.png)
![alt text](https://cdn-images-1.medium.com/max/1600/1*05ExQKJ0nOoWV80SNVEyJg.png)


Our Neural Network, based heavily on Andrej’s solution, will do the following:

1. Take in images from the game and preprocess them (remove color, background, downsample etc.).
2. Use the Neural Network to compute a probability of moving up.
3. Sample from that probability distribution and tell the agent to move up or down.
4. If the round is over (you missed the ball or the opponent missed the ball), find whether you won or lost.
5. When the episode has finished(someone got to 21 points), pass the result through the backpropagation algorithm to compute the gradient for our weights.
6. After 10 episodes have finished, sum up the gradient and move the weights in the direction of the gradient.
7. Repeat this process until our weights are tuned to the point where we can beat the computer. That’s basically it! Let’s start looking at how our code achieves this.

In [0]:
#Imports
import gym
import numpy as np

In [0]:
#Make the environment and get the first image of the game
env         = gym.make("Pong-v0")
observation = env.reset()

#HyperParameters

#How many episodes to wait before moving the weights
batch_size               = 10  
#Discount factor to discount the effect of old actions on the final result
gamma                    = 0.99
#Parameter used in RMSProp algorithm
decay_rate               = 0.99
#Hoy many neurons ar in hidden layer
num_hidden_layer_neurons = 200
#Dimensions of our observation image
input_dimensions         = 80*80
#The rate at which we learn from our results to compute the new weights
#A higher rate means we react more to results and a lower rate means 
#we don't react as strongly to each result
learning_rate            = 1e-4

episode_number             = 0
reward_sum                 = 0
running_reward             = None
prev_processed_observation = None

#Initialize Weights

In [0]:
#Layers

#               Dimension            Result
#Observations  6400x1    Matrix
#Layer 1       200x6400  Matrix      200x1
#Layer 2       1x200     Matrix      1x1

#Initialize Weights
weights = {
    '1': np.random.randn(num_hidden_layer_neurons, input_dimensions) / np.sqrt(input_dimensions)         ,
    '2': np.random.randn(num_hidden_layer_neurons)                   / np.sqrt(num_hidden_layer_neurons)
}


#Initialize RMS Props Parameters and Historic Values´ Containers

In [0]:
#Initial Parameters for RMSProp Algorithm
#(http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop)
expectation_g_squared     = {}
g_dict                    = {}
for layer_name  in weights.keys():
    expectation_g_squared[layer_name] = np.zeros_like(weights[layer_name])
    g_dict[layer_name]                = np.zeros_like(weights[layer_name])

#Array to collect Observations and Intermidiate Values across to
#compute the gradient at the end based on the result
episode_hidden_layer_values,    \
episode_onservations,           \
episode_gradient_log_ps,        \
episode_rewards = [],[],[],[]

#Pre-processed the Image Frames

Let’s dive into preprocess_observations to see how we convert the image OpenAI Gym gives us into something we can use to train our Neural Network. The basic steps are:

1. Crop the image (we just care about the parts with information we care about).
2. Downsample the image. (Shrinking)
3. Convert the image to black and white (color is not particularly important to us).
4. Remove the background.
5. Convert from an 80 x 80 matrix of values to 6400 x 1 matrix (flatten the matrix so it’s easier to use).
6. Store just the difference between the current frame and the previous frame if we know the previous frame (we only care about what’s changed).

In [0]:
def downsample(image):
  #Take only alternate pixels 
  #Basicaly halves the resolution of the image (which is fine for us)
  return image[::2, ::2, :]

def remove_color(image):
  #Convert all color (RGB is the third dimension in the image)
  return image[:, :, 0]

def remove_background(image):
  image[image == 144] = 0
  image[image == 109] = 0
  return image

def preprocess_observation(input_observation, prev_observation, input_dimension):
    """Convert the 210x160x3 uint8 frame into a 6400 flout vector"""
    processed_observation = input_observation[35:195]                  #Crop
    processed_observation = downsapme(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    
    #Set everything left to 1 (paddles, ball)
    processed_observation[processed_observation != 0] = 1
    
    #Substract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
      input_observation = processed_observation - prev_processed_observation
    else:
      input_observation = np.zeros(input_dimensions)
    
    #Store the previous frame so we can substract from it next time
    prev_processed_observation = processed_observation
    
    return input_observation, prev_processed_observation

#Forward Propagation


1. Compute the unprocessed hidden layer values by simply finding the dot product of the weights[1] (weights of layer 1) and the observation_matrix. We have 200 neurons so each row represents the output of one neuron.

  **HiddenLayer(200x1) = W1(200x6400) * ObservationMatrix(6400x1) **

2. Next, we apply a non linear thresholding function on those hidden layer values - in this case just a simple ReLU. At a high level, this introduces the nonlinearities that makes our network capable of computing nonlinear functions rather than just simple linear ones.

3. We use those hidden layer activation values to calculate the output layer values. 

  **OutputLayer(1x1) = W1(1x200) * HiddenLayer(200x1) **

4. Finally, we apply a sigmoid function on this output value so that it’s between 0 and 1 and is therefore a valid probability (probability of going up).

![Matrix Multiplication.](pong_assets/2.png)
![Matrix Multiplication.](pong_assets/3.png)

In [0]:
def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

def relu(vector):
  vector[vector < 0] = 0
  return vector

def apply_neural_nets(observation_matrix, weights):
  ##Based on the obsevation_matrix and weights, compute the new hidden layer values and the new output layer values
  hidden_layer_values = np.dot(weights['1'], observation_matrix)
  hidden_layer_values = relu(hidden_layer_values)
  hidden_layer_values = np.dot(hidden_layer_values)
  
  output_layer_values = np.dot(hidden_layer_values, weights['2'])
  output_layer_values = sigmoid(output_layer_values)
  return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3
      

#Discount the Rewards

If you moved up at the first frame of the episode, it probably had very little impact on whether or not you win. However, **closer to the end of the episode, your actions probably have a much larger effect as they determine whether or not your paddle reaches the ball and how your paddle hits the ball.**

In [0]:
def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_reward

#Learning


Learning is all about seeing the result of the action (i.e. whether or not we won the round) and changing our weights accordingly. The first step to learning is asking the following question:


**How does changing the output probability (of going up) affect my result of winning the round?**
Mathematically, this is just the derivative of our result with respect to the outputs of our final layer. If **L** is the value of our result to us and **f** is the function that gives us the activations of our final layer, this derivative is just **∂L/∂f.**


In a binary classification context (i.e. we just have to tell the AI one of two actions, up or down), this derivative turns out to be

![alt text](https://image.ibb.co/g5eEAT/image.png)

Note that **σ** in the above equation represents the sigmoid function. Read the Attribute Classification section here:

http://cs231n.github.io/neural-networks-2/#losses

for more information about how we get the above derivative. We simplify this further below:

**∂L/∂f = true_label(0 or 1) — predicted_label(0 or 1)**



After one action(moving the paddle up or down), we don’t really have an idea of whether or not this was the right action. So we’re going to cheat and treat the action we end up sampling from our probability as the correct action.


After calculating gradient per action, the next step is to figure out how we learn after the end of an episode (i.e. when we or our opponent miss the ball and someone gets a point). We do this by computing the policy gradient of the network at the end of each episode. The intuition here is that if we won the round, we’d like our network to generate more of the actions that led to us winning. Alternatively, if we lose, we’re going to try and generate less of these actions.

OpenAI Gym provides us the handy done variable to tell us when an episode finishes (i.e. we missed the ball or our opponent missed the ball). When we notice we are done, the first thing we do is compile all our observations and gradient calculations for the episode. This allows us to apply our learnings over all the actions in the episode.

**Use backpropagation to compute the gradient (i.e. the direction we need to move our weights to improve).**


As you’ll see in that excerpt, there are four fundamental equations of backpropogation, a technique for computing the gradient for our weights.


![alt text](https://image.ibb.co/fouRQT/image.png)


Our goal is to find ∂C/∂w1 (BP4), the derivative of the cost function with respect to the first layer’s weights, and ∂C/∂w2, the derivative of the cost function with respect to the second layer’s weights. These gradients will help us understand what direction to move our weights in for the greatest improvement.

To begin with, let’s start with **∂C/∂w2**. If **a^l2** is the activations of the hidden layer (layer 2), we see that the formula is:

![alt text](https://image.ibb.co/cjWHX8/image.png)

Next, we need to calculate ∂C/∂w1. The formula for that is:

![alt text](https://image.ibb.co/bLSKeo/image.png)


So all we need now is δ^l2. Once we have that, we can calculate ∂C/∂w1 and return.


http://neuralnetworksanddeeplearning.com/chap2.html




In [0]:
def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
  """See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
  delta_L = gradient_log_p
  dC_dw2  = np.dot(hidden_layer_values.T, delta_L).ravel()
  
  delta_l2 = np.outer(delta_L, weights['2'])
  delta_l2 = relu(delta_l2)
  dC_dw1   = np.dot(delta_l2.T, observation_values)
  
  return {
      '1': dC_dw1,
      '2': dC_dw2
  }


#Update Weights

To update the weights, we simply apply RMSProp, an algorithm for updating weights described by Sebastian Reuder here.

![alt text](https://image.ibb.co/eQFuUo/image.png)

http://ruder.io/optimizing-gradient-descent/index.html#rmsprop

In [0]:
def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
  """See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
  epsilon = 1e-5
  
  for layer_name in weights-keys():
    g                                 =  g_dict[layer_name] 
    expectation_g_squared[layer_name] =  decay_rate * expectation_g_squared[layer_name] + ( 1 - decay_rate ) * g**2
    weights[layer_name]               += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
    
    #Reset Batch gradient Buffer
    g_dict[layer_name]                =  np.zeros_like(weights[layer_name])                                         

In [0]:
while True:
    #Pre-process the image
    env.render()
    processed_observations, prev_processed_observation /
    = preprocess_observations(
        observation,
        prev_processed_observations,
        input_dimensions
    )
    
    #Send the processed observation through the policy network (neural network)
    #To generate the probability of telling the AI to move up
    hidden_layer_values, up_probability = apply_neural_nets(processed_observation, weights)
    episode_observations.append(processed_observations)
    episode_hidden_layer_values-append(hidden_layer_values)
    
    #Choose an action by flipping an imaginary coin
    action = choose_action(up_probability)
    
    #Carry out the action
    observation, reward, done, info = env.step(action)
    
    #Record the results for later learning
    reward_sum     += reward
    episode_rewards.append(reward)
    
    #More info:  http://cs231n.github.io/neural-networks-2/#losses
    #Gradient per action
    fake_label = 1 if action == 2 else 0
    loss_function_gradient = fake_label - up_probability
    episode_gradient_log_ps.append(loss_function_gradient)
    
    #Check if the episode is finished
    if done:
      episode_number += 1
      
      """
      Figure out how we learn after the end of and episode. We do this
      by computing the policy gradient of the network at the end of each episode. 
      
      Compile all our observations and gradient calculations for the episode. 
      This allows us to apply our learnings over all the actions in the episode.
      """
      episode_hidden_layer_values = np.vstack(episode_hidden_layer_values)
      episode_observations        = np.vstack(episode_observations)
      episode_gradient_log_ps     = np.vstack(episode_gradient_log_ps)
      episode_rewards             = np.vstack(episode_rewards)
      
      """
      Next, we want to learn in such a way that actions taken towards the end of 
      an episode more heavily influence our learning than actions taken at the beginning. 
      This is called discounting.
      """
      episode_gradient_log_ps_discounted = discount_with_reward(
          episode_gradient_log_ps,
          episode_rewards,
          gamma
      )
      gradient = compute_gradient(
          episode_gradient_log_ps_discounted,
          episode_hidden_layer_values,
          episode_observations,
          weights
      )
      
      # Sum the gradient for use when we hit the batch size
      for layer_name in gradient:
          g_dict[layer_name] += gradient[layer_name]
      
      #Update weights for our Neural Network and implement our learning
      if episode_number % batch_size == 0:
          update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate)

      episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], [] # reset values
      observation = env.reset() # reset env
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
      reward_sum = 0
      prev_processed_observations = None
      
    
    
    
    
    

SyntaxError: ignored

Save weights to Google drive
https://colab.research.google.com/notebooks/io.ipynb
